This is the script used to produce embeddings of paradigms in Chapter 6 of my PhD dissertation. It takes as input a .csv file and produces an Excel spreadsheet containing an embedding for each individual language.

In [ ]:
import pandas as pd
import numpy as np
import os

from pathlib import Path, PureWindowsPath

import openai

It is assumed that the .csv file is formatted using at least 3 columns - 'Lang', 'Key', 'Value'

In [ ]:
df = pd.read_csv("inputfile.csv")
print(df)

In [ ]:
gkk = df.groupby('Lang')['Value'].value_counts()

gkdf = df.groupby('Lang')['Value'].nunique()
gkdf = gkdf.to_frame()
print(gkdf)

gkhist = gkdf['Value'].value_counts()
gkhistx = [gkhist]
#print(gkhist)


In [ ]:
df['FullPatt'] = df.groupby('Lang')["Value"].transform(lambda x: ' '.join((map(str, x))))
dfdd = df['FullPatt'].drop_duplicates()
dfdl = df['Lang'].drop_duplicates()
print(dfdd)
print(dfdl)

In [ ]:
prevec=open('label.txt', 'w', encoding='utf-8')
for item in dfdl:
    prevec.write("%s\n" % item)
prevec.close()

In [ ]:
ddf = pd.DataFrame({'Lang': dfdl, 'ProString': dfdd})
print(ddf)

In [ ]:
dfvec=pd.DataFrame()

openai.api_key = "insert key here"

for lang, x in ddf.values:

    embedding = openai.Embedding.create(input=str(x), model="text-embedding-3-small")["data"][0]["embedding"]
   
    lengthvec = len(embedding)
 
    dfvec = dfvec.append(pd.Series(embedding), ignore_index=True)
    
dfvec.apply(pd.to_numeric)

In [ ]:
df_vecs = pd.DataFrame(dfvec)
df_vecs.to_excel('output.xlsx', index=False, header=False)

In [ ]:
langlist=dfdl.tolist()

In [ ]:
df_vecs.insert(0, "Lang", langlist)
df_vecs.set_index("Lang", inplace = True)
#print(dataVecs)
print(df_vecs)

In [ ]:
df_vecs.to_excel("vector output file.xlsx", index=True, header=False)